In [61]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,Conv1D, Flatten, MaxPooling2D,MaxPooling1D, Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
import pickle
from keras.optimizers import Adam
import csv
from sklearn.metrics import confusion_matrix
import time
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Bidirectional, LSTM

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
data = pd.read_csv("/content/drive/MyDrive/traffic/tfidf_embeddings.csv")
data

,00pm,10a,10th,125th,12pm,145th,14th,16th,180th,19th,...,yes,yet,york,you,youd,your,youre,youth,zone,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,2
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.184964,0.0,0.200696,0.0,0.0,0.0,0


In [64]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.60, test_size = 0.40, shuffle=False, random_state = 0)

In [65]:
# Convert labels to categorical format
NoClass = 3  # Number of classes
y_train_categorical = to_categorical(y_train, num_classes=NoClass)
y_test_categorical = to_categorical(y_test, num_classes=NoClass)


In [66]:
# Reshape the data for the Conv2D layer
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1, 1)  # Assuming your data is 2D (samples, features)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1, 1)

In [67]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(NoClass, activation='softmax'))

In [68]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test_reshaped, y_test_categorical), verbose=1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
29/29 [==============================] - 123s 4s/step - loss: 1.0447 - accuracy: 0.4300 - val_loss: 0.8015 - val_accuracy: 0.6675
Epoch 2/50
29/29 [==============================] - 102s 4s/step - loss: 4.6751 - accuracy: 0.4000 - val_loss: 0.8623 - val_accuracy: 0.5233
Epoch 3/50
29/29 [==============================] - 100s 3s/step - loss: 0.7919 - accuracy: 0.6706 - val_loss: 0.7491 - val_accuracy: 0.7392
Epoch 4/50
29/29 [==============================] - 102s 3s/step - loss: 0.7297 - accuracy: 0.7506 - val_loss: 0.6850 - val_accuracy: 0.7492
Epoch 5/50
29/29 [==============================] - 92s 3s/step - loss: 0.6747 - accuracy: 0.7472 - val_loss: 0.6426 - val_accuracy: 0.7517
Epoch 6/50
29/29 [==============================] - 102s 4s/step - loss: 0.6397 - accuracy: 0.7550 - val_loss: 0.6211 - val_accuracy: 0.7592
Epoch 7/50
29/29 [==============================] - 100s 3s/step - loss: 0.6277 - accuracy: 0.7550 - val_loss: 0.5918 - val_accuracy: 0.7833
Epoch 8/50
29/

In [69]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train_categorical)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

57/57 [==============================] - 26s 462ms/step - loss: 0.0197 - accuracy: 0.9978
Train Accuracy: 99.78%
Test Accuracy: 91.42%
